In [98]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


# Getting Dataframes

In [99]:
master_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
train = master_train
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [100]:
master_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
test = master_test
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


### Data Wrangling

In [101]:
train.shape

(1460, 81)

In [102]:
#train.info(verbose=True)

In [103]:
#pd.set_option('display.max_rows',None)
#train.isna().sum()

In [104]:
train['MiscVal'].unique()

array([    0,   700,   350,   500,   400,   480,   450, 15500,  1200,
         800,  2000,   600,  3500,  1300,    54,   620,   560,  1400,
        8300,  1150,  2500])

In [105]:
train[['MiscFeature','MiscVal']]

,MiscFeature,MiscVal
0,NaN,0
1,NaN,0
2,NaN,0
3,NaN,0
4,NaN,0
...,...,...
1455,NaN,0
1456,NaN,0
1457,Shed,2500
1458,NaN,0


In [106]:
train = train.drop(['MiscFeature'], axis=1)
test = test.drop(['MiscFeature'], axis=1)

In [107]:
print("shape of train data", train.shape)
print("shape of test data", test.shape)

shape of train data (1460, 80)
shape of test data (1459, 79)


## Find Z-score for outliers

get numeric data only

In [108]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
temp = train.select_dtypes(include=numerics)
#temp.head()

remove columns having null values

In [109]:
#temp.isna().sum()>0

In [110]:
temp = temp.drop(['LotFrontage', 'GarageYrBlt', 'MasVnrArea'], axis=1)
temp.head()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,8450,7,5,2003,2003,706,0,150,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,9600,6,8,1976,1976,978,0,284,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,11250,7,5,2001,2002,486,0,434,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,9550,7,5,1915,1970,216,0,540,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,14260,8,5,2000,2000,655,0,490,...,192,84,0,0,0,0,0,12,2008,250000


In [111]:
from scipy.stats import zscore

print ("shape with outliers", temp.shape)
print ("shape after outliers", temp[(np.abs(stats.zscore(temp)) < 3).all(axis=1)].shape)

shape with outliers (1460, 35)
shape after outliers (1037, 35)


There is large amount of outliers. Almost one third of dataset so we will leave them

# Feature Engineering

## Separating independent and dependent variables

In [112]:
X_train=train
X_train=X_train.drop(['SalePrice'], axis=1)
X_test=test
X_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,0,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,0,NaN,NaN,0,12,2008,WD,Normal


In [113]:
Y_train=train['SalePrice']
Y_train.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

## One hot encoding

In [114]:
print("shape before one hot encoding", X_train.shape)
X_train=pd.get_dummies(X_train)

dummies_frame = pd.get_dummies(X_train)
X_test=X_test.reindex(columns = dummies_frame.columns, fill_value=0)
#X_test=pd.get_dummies(X_test)

print("shape after one hot encoding", X_train.shape)
X_train.head()

shape before one hot encoding (1460, 79)
shape after one hot encoding (1460, 285)


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0


In [115]:
X_test.shape

(1459, 285)

In [116]:
X_train=X_train.fillna(0)
X_test=X_test.fillna(0)
#X.isna().sum()

In [117]:
print("shape of X_train", X_train.shape)
print("shape of X_test", X_test.shape)

shape of X_train (1460, 285)
shape of X_test (1459, 285)


## Data Scaling

In [118]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)
X_scaled_train

array([[-1.73086488,  0.07337496,  0.2128772 , ..., -0.11785113,
         0.4676514 , -0.30599503],
       [-1.7284922 , -0.87256276,  0.64574726, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [-1.72611953,  0.07337496,  0.29945121, ..., -0.11785113,
         0.4676514 , -0.30599503],
       ...,
       [ 1.72611953,  0.30985939,  0.2417352 , ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 1.7284922 , -0.87256276,  0.29945121, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 1.73086488, -0.87256276,  0.50145724, ..., -0.11785113,
         0.4676514 , -0.30599503]])

In [119]:
np.any(np.isnan(X_scaled_train))

False

## Dimensionality Reduction using PCA

In [120]:
from sklearn.decomposition import PCA

pca = PCA(0.95)
X_pca = pca.fit_transform(X_scaled_train)
X_pca.shape

(1460, 181)

# Ridge Regression

In [124]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

errors=[]

sweep = np.arange(0, 5, 1, dtype=int)
for i in sweep:
    poly = PolynomialFeatures(degree = i)
    X_poly=poly.fit_transform(X_scaled_train)
    ridge = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1], cv=4)
    ridge.fit(X_poly, Y_train)
    errors.append(ridge.score(X_poly, Y_train))
    
print(errors)

ValueError: could not broadcast input array from shape (1460,285) into shape (1460,0)

## Submission

In [48]:
predictions = ridge.predict(X_scaled_test)
print(predictions)

[ 85698.62377049 122775.21392917 152097.91382041 ... 143349.50266089
  81530.35257763 197759.0456291 ]


In [52]:
X_test['SalePrice']=predictions
#X_test.head()

In [63]:
submit = X_test[['Id','SalePrice']].set_index('Id')
Id = X_test[["Id"]].to_numpy()
SalePrice = X_test[["SalePrice"]].to_numpy()
submit.head()

,SalePrice
Id,
1461,85698.623770
1462,122775.213929
1463,152097.913820
1464,160320.625236
1465,127745.549168


In [84]:
!cd /kaggle/working
submit.to_csv("submission.csv")
!head submission.csv

Id,SalePrice
1461,85698.62377048585
1462,122775.2139291711
1463,152097.91382040922
1464,160320.62523626236
1465,127745.54916827063
1466,141600.50100826676
1467,139108.87343639255
1468,126855.09131028436
1469,142171.73343868396


In [64]:
SalePrice.shape

(1459, 1)